# 3 - B: Training a Word2Vec Model

## 목표
- 한국어 단어 임베딩 벡터를 Word2Vec을 학습시켜 얻어봅시다.  
- 학습데이터로는 [네이버 영화리뷰 데이터](https://drive.google.com/file/d/1eEWFL-vKGP5pReH8dxD1KvtR6f0BfTqS/view?usp=sharing)를 사용해볼 것입니다.

In [1]:
# install the libraries needed
!pip3 install gensim  # word2vec 훈련을 위해
!pip3 install pandas # 데이터 시각화를 위해
!pip3 install konlpy  # 한국어 형태소 분석을 위해

     |████████████████████████████████| 19.4 MB 1.4 MB/s 
     |████████████████████████████████| 448 kB 69.2 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
# import the libraries needed
from gensim.models.word2vec import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec  # 매 에폭마다의 로스를 확인하기 위해.
from konlpy.tag import Okt
from typing import List
import pandas as pd
from tqdm import tqdm
import gdown
# --- Word2Vec 훈련이 진행되는 과정을 모니터링 하기 위해, 모든 로깅 레벨을 INFO로 올려줍니다. --- #
from sys import stdout
import logging
logging.basicConfig(stream=stdout, level=logging.DEBUG)

In [3]:
# --- 데이터 다운로드를 위한 코드; 수정하지 말아주세요! --- #
NAVER_REVIEWS_TSV_URL = "https://drive.google.com/u/0/uc?id=1eEWFL-vKGP5pReH8dxD1KvtR6f0BfTqS&export=download"
NAVER_REVIEWS_TSV = "./naver_reviews.tsv"
# 드라이브에 업로드한 (data/naver_reviews.tsv) 파일을 로컬에 다운로드
gdown.download(url=NAVER_REVIEWS_TSV_URL, output=NAVER_REVIEWS_TSV, quiet=False)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): drive.google.com:443
DEBUG:urllib3.connectionpool:https://drive.google.com:443 "GET /u/0/uc?id=1eEWFL-vKGP5pReH8dxD1KvtR6f0BfTqS&export=download HTTP/1.1" 302 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): doc-0o-08-docs.googleusercontent.com:443
DEBUG:urllib3.connectionpool:https://doc-0o-08-docs.googleusercontent.com:443 "GET /docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/2oqvfu5e43fdvcqs1br1rr0irh0d4v26/1629697575000/11244659521636006499/*/1eEWFL-vKGP5pReH8dxD1KvtR6f0BfTqS?e=download HTTP/1.1" 200 None


Downloading...
From: https://drive.google.com/u/0/uc?id=1eEWFL-vKGP5pReH8dxD1KvtR6f0BfTqS&export=download
To: /content/naver_reviews.tsv
19.5MB [00:00, 78.0MB/s]


'./naver_reviews.tsv'

In [4]:
# 네이버 영화 데이터를 pandas dataframe으로 로드합니다. 구분자가 \t 인 tsv 파일을 읽어야 하니, sep="\t"로 넣어줍니다.
reviews_df = pd.read_csv(NAVER_REVIEWS_TSV, sep="\t") #tsv file tab 을 기준으로 구분하는 문자 이다 . 

In [5]:
# (id, 리뷰, 긍정=1/부정=0)으로 구성된 데이터입니다.
reviews_df.head(10)  # 상위 10개 문서 출력

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
5,2190435,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1
6,9279041,완전 감동입니다 다시봐도 감동,1
7,7865729,개들의 전쟁2 나오나요? 나오면 1빠로 보고 싶음,1
8,7477618,굿,1
9,9250537,바보가 아니라 병 쉰 인듯,1


In [6]:
# 각 모든 데이터 중에, 결측치가 하나라도 존재하는지 확인해볼게요.
# df.info(): https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html
print(reviews_df.isnull().values.any())
reviews_df.info()

True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        200000 non-null  int64 
 1   document  199992 non-null  object
 2   label     200000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [7]:
# document 컬럼을 보아하니, 결측치가 조금 있습니다. 결측치를 제거하기 위해 dropna()를 사용하겠습니다.
# df.dropna() 문서: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
# how = any 로 설정할 경우? null인 컬럼이 하나라도 있으면 해당 row를 드랍.
# how = all 로 설정할 경우? 모든 컬럼이 null 일 경우에만 해당 row를 드랍.
reviews_df = reviews_df.dropna(how="any")
print(reviews_df.isnull().values.any())
reviews_df.info()

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
False
<class 'pandas.core.frame.DataFrame'>
Int64Index: 199992 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        199992 non-null  int64 
 1   document  199992 non-null  object
 2   label     199992 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 6.1+ MB


In [8]:
# ---- 전체 리뷰문서 토크나이징을 진행합니다 --- # 
okt = Okt()  # okt 형태소 분석기를 토크나이저로 사용합니다.
corpus: List[List[str]] = list()
# 모든 데이터를 다 토크나이즈를 하기에는 시간이 많이 걸릴 것입니다. 
# 앞 10만개 정도만 선택하여 토크나이즈를 해보도록 하겠습니다.
reviews_df = reviews_df[:100000] 
for sent in tqdm(reviews_df['document'], "tokenization in progress..."):
    # 토큰화를 진행할 때, 어근을 추출하여 토큰화 동시에 정규화를 해줍니다 (e.g. 밝은 -> 밝다)
    tokens = okt.morphs(sent, stem=True)  #stem = True 어간도 같이 추출 한다.
    corpus.append(tokens)

tokenization in progress...: 100%|██████████| 100000/100000 [04:19<00:00, 385.83it/s]


In [9]:
# --- 전처리된 말뭉치 확인 --- #
# 대부분의 단어들이 정규화되어, "-다"로 끝나는 것을 확인해볼 수 있습니다. 
for tokens in corpus[:10]:
  print(tokens) 

['어리다', '때', '보고', '지금', '다시', '보다', '재밌다', 'ㅋㅋ']
['디자인', '을', '배우다', '학생', '으로', ',', '외국', '디자이너', '와', '그', '들', '이', '일군', '전통', '을', '통해', '발전', '하다', '문화', '산업', '이', '부럽다', '.', '사실', '우리나라', '에서도', '그', '어렵다', '시절', '에', '끝', '까지', '열정', '을', '지키다', '노라노', '같다', '전통', '이', '있다', '저', '와', '같다', '사람', '들', '이', '꿈', '을', '꾸다', '이루다', '나가다', '수', '있다', '것', '에', '감사하다', '.']
['폴리스스토리', '시리즈', '는', '1', '부터', '뉴', '까지', '버리다', '하나', '도', '없다', '..', '최고', '.']
['오다', '..', '연기', '가', '진짜', '개', '쩔다', '..', '지루하다', '생각', '하다', '몰입', '하다', '보다', '..', '그렇다', '이렇다', '진짜', '영화', '지']
['안개', '자욱하다', '밤하늘', '에', '뜨다', '있다', '초승달', '같다', '영화', '.']
['사랑', '을', '해보다', '사람', '이', '라면', '처음', '부터', '끝', '까지', '웃다', '있다', '영화']
['완전', '감동', '이다', '다시', '보다', '감동']
['개', '들', '의', '전쟁', '2', '나오다', '?', '나오다', '1', '빠', '로', '보고', '싶다']
['굿']
['바보', '가', '아니다', '병', '쉰', '이다']


In [10]:
# --- 워드 투 벡터의 훈련상황을 모니터링하기 위해, 메 에폭마다 모델의 로스 (error)를 출력해볼 수 있습니다. --- #
# 이를 위해서 Callback이라는 객체를 사용할 수 있습니다. 
# https://stackoverflow.com/a/54891714
class LossCallBack(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
        self.losses = list()  # collect losses here
        self.losses.append(0.0)

    def on_epoch_end(self, model):
        """
        이 함수는 매 에폭의 마지막 단계에 호출됩니다.
        """
        # get_ latest_training_loss는 현재까지의 cumulative loss를 출력합니다.
        loss = model.get_latest_training_loss()
        self.epoch += 1
        self.losses.append(loss)
        # 현재까지의 축적된 로스의 총합을 리포트
        print('Cumulative loss after epoch {}: {}'.format(self.epoch, loss))
        # 바로 이전 로스와의 차이도 리포트
        print('Offset to previous loss: {}'.format(str(self.losses[-1] - self.losses[-2])))

In [12]:
# --- TODO 1 --- # 
# Word2Vec의 객체를 생성하면, 바로 Word2Vec 모델의 학습이 진행됩니다. 
# Word2Vec 클래스 문서를 참고하여: https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec
# Word2Vec 모델학습을 시작해보세요!
# 하이퍼파라미터는 다음과 같이 설정해주세요:
# alpha (learning rate) = 0.01
# window (윈도우의 길이) = 6
# min_count (학습할 단어의 최소 빈도수) = 3
# sg (skipgram 사용여부) =  1
# iter (에폭) = 50
# compute_loss (매 에폭마다 로스 계산) = True
# callbacks (매 에폭마다 호출할 것들) = (LossCallBack(),)
model = Word2Vec(sentences=corpus #말뭉치
                 ,alpha=0.01, #learning rate
                 window=6,  #윈도우의 크기 (ngrams 의 n)
                 min_count=3, #embedding 을 구한 단어의 최소 빈도수 
                 sg=1, #sg = 0 cbow sg=1 skipgram
                 iter=50,  #epoch
                 compute_loss=True, #매 에폭 마다 LOSS 계산 
                 negative = 5,
                 callbacks=(LossCallBack(),)) #매 에폭마다 수행할 일을 넣어줄수 있다.
# ------------ #

INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 139027 words, keeping 12618 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #20000, processed 281908 words, keeping 18275 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #30000, processed 420918 words, keeping 22279 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #40000, processed 560545 words, keeping 25579 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #50000, processed 700594 words, keeping 28656 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #60000, processed 841566 words, keeping 31336 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #70000, processed 982494 words, keeping 33685 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #80000, processed 1124238 words, keeping 

In [13]:
model.wv.vocab

{'어리다': <gensim.models.keyedvectors.Vocab at 0x7f864dde7fd0>,
 '때': <gensim.models.keyedvectors.Vocab at 0x7f864dde7050>,
 '보고': <gensim.models.keyedvectors.Vocab at 0x7f864dde7ed0>,
 '지금': <gensim.models.keyedvectors.Vocab at 0x7f864dde7d90>,
 '다시': <gensim.models.keyedvectors.Vocab at 0x7f864ddd3e10>,
 '보다': <gensim.models.keyedvectors.Vocab at 0x7f864ddd3e50>,
 '재밌다': <gensim.models.keyedvectors.Vocab at 0x7f864ddd3d90>,
 'ㅋㅋ': <gensim.models.keyedvectors.Vocab at 0x7f864ddd3150>,
 '디자인': <gensim.models.keyedvectors.Vocab at 0x7f864dde7710>,
 '을': <gensim.models.keyedvectors.Vocab at 0x7f864ddd3f90>,
 '배우다': <gensim.models.keyedvectors.Vocab at 0x7f864ddd3210>,
 '학생': <gensim.models.keyedvectors.Vocab at 0x7f864ddd35d0>,
 '으로': <gensim.models.keyedvectors.Vocab at 0x7f864ddd37d0>,
 ',': <gensim.models.keyedvectors.Vocab at 0x7f864ddd3750>,
 '외국': <gensim.models.keyedvectors.Vocab at 0x7f864ddd3b10>,
 '디자이너': <gensim.models.keyedvectors.Vocab at 0x7f864ddd3ad0>,
 '와': <gensim.models.

In [14]:
model.wv.most_similar("설경구")

INFO:gensim.models.keyedvectors:precomputing L2-norms of word weight vectors


[('문소리', 0.6496538519859314),
 ('로빈윌리엄스', 0.6354153156280518),
 ('문정희', 0.6305896043777466),
 ('안성기', 0.6219744682312012),
 ('패닝', 0.6166368722915649),
 ('송윤아', 0.6034888625144958),
 ('주지훈', 0.6031726598739624),
 ('서영희', 0.5982471704483032),
 ('천우희', 0.5973948240280151),
 ('추상미', 0.5968494415283203)]

In [16]:
# 한석규는 어떤 배우인가? 
# 원하는 단어가 어휘에 포함되었는지를 확인하고 싶을 땐, model.wv.vocab을 접근하면 됩니다.
if "한석규" in model.wv.vocab: 
  for word, score in model.wv.most_similar("한석규"):
    print(word, score)
else:
  raise ValueError("out of vocab")

윤제문 0.6487352848052979
오달수 0.63154536485672
송강호 0.6285011172294617
임시완 0.6165424585342407
정려원 0.6142942905426025
차승원 0.6134588718414307
이제훈 0.6108124852180481
심혜진 0.6050857305526733
정보석 0.6043567657470703
황정민 0.6013203263282776


In [ ]:
# 어간 추출을 진행했기 때문에, "재밌어"라는 단어도 "재밌다"로 정규화되었을 것입니다.
if "재밌다" in model.wv.vocab:
  for word, score in model.wv.most_similar("재밌다"):
    print(word, score)
else:
  raise ValueError("out of vocab")

재미있다 0.9388811588287354
재다 0.843007504940033
재밋 0.6832064986228943
쟈밋 0.6739248037338257
재밋는듯 0.6704350113868713
재밋었습니 0.6672142148017883
잼남 0.6625919342041016
재밋어용 0.6621266007423401
토탈리콜 0.6534591317176819
재밋어 0.6479905843734741


## 질문거리 (reverse-engineering)

인물 명사 와 형용사는 왜 유사도 에서 나오지 않는가 
인물은 같은 윈도우에서  님 ,연기 인물을 꾸며주는 말이 나오고 다른 인물 명사에서도 



In [17]:
if "인도영화" in model.wv.vocab:
  for word, score in model.wv.most_similar("인도영화"):
    print(word, score)
else:
  raise ValueError("out of vocab")

뮤지컬영화 0.6471281051635742
영화 0.6291313171386719
굉장 0.6074318885803223
블랙홀 0.6020694971084595
애니메이션영화 0.5967851877212524
뿐더러 0.5894656777381897
특색 0.5843429565429688
까다롭다 0.5787320137023926
벅스라이프 0.5784770846366882
아노하나 0.5782922506332397
